# **Connect-4 using Minimax Algorithm with Alpha-Beta Pruning**

In [36]:
import numpy as np
import math
import random

ROW_COUNT = 6
COLUMN_COUNT = 7
WIN_COUNT = 4


In [37]:
def initialize_board():
    """
    Initialize the Connect-4 board.
    The board is represented as a 6x7 grid where:
      - 0 represents an empty space,
      - 1 represents the human player, and
      - 2 represents the AI player.
    """
    board = np.zeros((ROW_COUNT, COLUMN_COUNT), dtype=int)
    return board


In [38]:
def print_board(board):
    """
    Print the Connect-4 board in its raw form.
    Row 0 is printed at the top.
    """
    # Print column numbers on top for clarity
    print("Columns: " + " ".join(str(i) for i in range(COLUMN_COUNT)))
    # Print board as-is (row 0 at the top)
    for row in board:
        print(" ".join(str(cell) for cell in row))
    print()


In [39]:
def generate_moves(board):
    """
    Generate a list of valid columns where a piece can be dropped.
    A column is valid if the topmost space is empty.
    """
    valid_moves = [col for col in range(COLUMN_COUNT) if board[0][col] == 0]
    return valid_moves


In [40]:
def drop_piece(board, column, player):
    """
    Drop the piece (1 for human, 2 for AI) into the specified column.
    The piece will occupy the first available empty row in that column.
    """
    for row in range(ROW_COUNT-1, -1, -1):
        if board[row][column] == 0:
            board[row][column] = player
            break


In [41]:
def check_winner(board, player):
    """
    Check if the specified player has four in a row horizontally, vertically,
    or diagonally.
    """
    # Check horizontal locations
    for row in range(ROW_COUNT):
        for col in range(COLUMN_COUNT - 3):
            if all(board[row][col+i] == player for i in range(WIN_COUNT)):
                return True

    # Check vertical locations
    for col in range(COLUMN_COUNT):
        for row in range(ROW_COUNT - 3):
            if all(board[row+i][col] == player for i in range(WIN_COUNT)):
                return True

    # Check positively sloped diagonals
    for row in range(ROW_COUNT - 3):
        for col in range(COLUMN_COUNT - 3):
            if all(board[row+i][col+i] == player for i in range(WIN_COUNT)):
                return True

    # Check negatively sloped diagonals
    for row in range(3, ROW_COUNT):
        for col in range(COLUMN_COUNT - 3):
            if all(board[row-i][col+i] == player for i in range(WIN_COUNT)):
                return True

    return False


In [42]:
def evaluate_window(window, player):
    """
    Evaluate a window of four cells. Returns a score based on the count of
    player's and opponent's pieces.
    """
    score = 0
    opp_player = 1 if player == 2 else 2

    if window.count(player) == 4:
        score += 100
    elif window.count(player) == 3 and window.count(0) == 1:
        score += 5
    elif window.count(player) == 2 and window.count(0) == 2:
        score += 2

    if window.count(opp_player) == 3 and window.count(0) == 1:
        score -= 4

    return score


In [43]:
def evaluate(board, player):
    """
    Evaluate the board state for the given player. Positive scores favor the
    player while negative scores favor the opponent.
    """
    score = 0

    # Check terminal conditions
    if check_winner(board, player):
        return 100000
    opp_player = 1 if player == 2 else 2
    if check_winner(board, opp_player):
        return -100000

    # Score center column for its potential to form winning lines
    center_array = [int(i) for i in list(board[:, COLUMN_COUNT//2])]
    center_count = center_array.count(player)
    score += center_count * 3

    # Score horizontal windows
    for row in range(ROW_COUNT):
        row_array = [int(i) for i in list(board[row, :])]
        for col in range(COLUMN_COUNT - 3):
            window = row_array[col:col+WIN_COUNT]
            score += evaluate_window(window, player)

    # Score vertical windows
    for col in range(COLUMN_COUNT):
        col_array = [int(i) for i in list(board[:, col])]
        for row in range(ROW_COUNT - 3):
            window = col_array[row:row+WIN_COUNT]
            score += evaluate_window(window, player)

    # Score positive diagonal windows
    for row in range(ROW_COUNT - 3):
        for col in range(COLUMN_COUNT - 3):
            window = [board[row+i][col+i] for i in range(WIN_COUNT)]
            score += evaluate_window(window, player)

    # Score negative diagonal windows
    for row in range(ROW_COUNT - 3):
        for col in range(COLUMN_COUNT - 3):
            window = [board[row+3-i][col+i] for i in range(WIN_COUNT)]
            score += evaluate_window(window, player)

    return score


In [44]:
def is_terminal_node(board):
    """
    Return True if the board is in a terminal state (win for any player or full).
    """
    return check_winner(board, 1) or check_winner(board, 2) or len(generate_moves(board)) == 0


In [45]:
def minimax(board, depth, alpha, beta, maximizing_player):
    valid_moves = generate_moves(board)
    terminal = is_terminal_node(board)

    if depth == 0 or terminal:
        if terminal:
            if check_winner(board, 2):
                return (None, 100000)
            elif check_winner(board, 1):
                return (None, -100000)
            else:
                return (None, 0)
        else:
            return (None, evaluate(board, 2))

    if maximizing_player:
        value = -math.inf
        best_col = random.choice(valid_moves)
        for col in valid_moves:
            board_copy = board.copy()
            drop_piece(board_copy, col, 2)
            new_score = minimax(board_copy, depth-1, alpha, beta, False)[1]
            if new_score > value:
                value = new_score
                best_col = col
            alpha = max(alpha, value)
            if alpha >= beta:
                break  # Alpha-beta pruning
        return best_col, value
    else:
        value = math.inf
        best_col = random.choice(valid_moves)
        for col in valid_moves:
            board_copy = board.copy()
            drop_piece(board_copy, col, 1)
            new_score = minimax(board_copy, depth-1, alpha, beta, True)[1]
            if new_score < value:
                value = new_score
                best_col = col
            beta = min(beta, value)
            if alpha >= beta:
                break  # Alpha-beta pruning
        return best_col, value


In [46]:
def best_move(board):
    """
    Calculate the best move for the AI player using the Minimax algorithm.
    """
    col, _ = minimax(board, 4, -math.inf, math.inf, True)
    return col


In [47]:
def human_move(board):
    """
    Allow the human player to choose a valid move (column number).
    Validates that the column is within range and not full.
    """
    valid_moves = generate_moves(board)
    while True:
        try:
            col = int(input("Enter your move (0-6): "))
            if col in valid_moves:
                return col
            else:
                print("Invalid move. Valid columns are:", valid_moves)
        except ValueError:
            print("Please enter an integer value.")


In [48]:
def play_game():
    """
    Run the Connect-4 game simulation where the human player and AI take turns.
    """
    board = initialize_board()
    game_over = False
    turn = random.choice([0, 1])  # Randomly choose who starts: 0 for human, 1 for AI

    print_board(board)
    while not game_over:
        if turn == 0:
            print("Human's turn:")
            col = human_move(board)
            drop_piece(board, col, 1)
            if check_winner(board, 1):
                print_board(board)
                print("Human wins!")
                game_over = True
            turn = 1
        else:
            print("AI's turn:")
            col = best_move(board)
            if col is None:
                print("Game is a draw!")
                game_over = True
            else:
                drop_piece(board, col, 2)
                if check_winner(board, 2):
                    print_board(board)
                    print("AI wins!")
                    game_over = True
                turn = 0

        print_board(board)
        if len(generate_moves(board)) == 0:
            print("Game is a draw!")
            game_over = True


In [35]:
if __name__ == "__main__":
    play_game()


Columns: 0 1 2 3 4 5 6
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0

Human's turn:
Enter your move (0-6): 3
Columns: 0 1 2 3 4 5 6
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 1 0 0 0

AI's turn:
Columns: 0 1 2 3 4 5 6
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 2 1 0 0 0

Human's turn:
Enter your move (0-6): 3
Columns: 0 1 2 3 4 5 6
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 1 0 0 0
0 0 2 1 0 0 0

AI's turn:
Columns: 0 1 2 3 4 5 6
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 2 0 0 0
0 0 0 1 0 0 0
0 0 2 1 0 0 0

Human's turn:
Enter your move (0-6): 0
Columns: 0 1 2 3 4 5 6
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 2 0 0 0
0 0 0 1 0 0 0
1 0 2 1 0 0 0

AI's turn:
Columns: 0 1 2 3 4 5 6
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 2 0 0 0
0 0 2 1 0 0 0
1 0 2 1 0 0 0

Human's turn:
Enter your move (0-6): 2
Columns: 0 1 2 3 4 5 6
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 